<a href="https://colab.research.google.com/github/enjoyPG/2023Gifted/blob/main/students/jwoo428/20230307~20230308_%EC%9D%B4%EC%A4%80%EC%9A%B0_5_2%EA%B5%90%EC%B0%A8_%EA%B2%80%EC%A6%9D%EA%B3%BC_%EA%B7%B8%EB%A6%AC%EB%93%9C_%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#교차 검증과 그리드 서치

#핵심키워드
###검증세트
* 하이퍼파라미터 튜닝을 위해 모델을ㅇ 평가할 때, 테스트 세트를 사용하지 않기 위해 훈련세트에서 다시 떼어낸 데이터 세트
###교차검증
* 훈련세트를 여러 폴드로 나눈 다음 한 폴드가 검증 세트의 역할을 하고 나머지 폴드에서는 모델을 훈련한다.
* 이러한 방식으로 모든 폴드에 대해 검증점수를 얻어 평균을 내는 방법
###그리드 서치
* 하이퍼파라미터탐색을 자동화해주는 도구.
* 탐색할 매개변수를 나열하면 교차검증을 수행하여 가장 좋은 검증 점수의 매개변수 조합을 선택함.
* 마지막으로 해당 매개변수 조합으로 최종모델을 훈련함
###랜덤서치
* 연속된 매개변수 값을 탐색할 때 유용함.
* 탐색할 값을 직접 나열하는것이 아니고 탐색 값을 샘플링할 수 있는 확률 분포 객체를 전달함.
* 지정된 횟수만큼 샘플링하여 교차검증을 수행하기 때문에 시스템 자원이 허락하는 만큼 탐색량을 조절할 수 있음

#핵심 패키지와 함수
###cross_validate()
* 교차검증을 수행하는 함수
###GridSearchCV
* 교차검증으로 하이퍼파라미터 탐색을 수행
###RandomizedSearchCV
* 교차검증으로 랜덤한 하이퍼파라미터 탐색을 수행

###검증세트
* 테스트세트를 사용하지 않으면 모델이 과대적합인지, 과소적합인지 판단하기 어렵다. 테스트 세트를 사용하지 않고 이를 측정하기 위해 훈련세트를 또 나누는 방법이 있는데 이를 검증세트라고 한다

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [2]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2,random_state=42)
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [3]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


###교차검증
* 검증세트를 만드느라 훈련세트가 감소합니다.
* 교차검증을 통해 안정적인 검증점수를 받을 수 있게 합니다.

* 사이킷런의 cross_validate()라는 교차검증함수를 사용

In [4]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01348257, 0.01114583, 0.00916052, 0.00873232, 0.00871921]), 'score_time': array([0.00168777, 0.00141144, 0.00121355, 0.00161958, 0.00134468]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


* 키에 담긴 5개점수를 평균하여 얻을 수 있음

In [5]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


* cross_validate()함수는 훈련세트를 섞지 않기 때문에 분할기를 지정해야 합니다.
* cross_validate()함순는 회귀모델일 경우 KFold 분할기를 사용하고 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold를 사용합니다.

In [6]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [7]:
splitter = StratifiedKFold(n_splits=10,shuffle=True, random_state=42)#n_splits 매개변수는 몇(k)폴드 교차 검증을 할지 결정함
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


###하이퍼파라미터 튜닝
* 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터를 하이퍼파라미터라고 합니다.


* 결정트리 모델에서 최적의 max_depth값을 찾아도 max_depth의 최적값은 min_samples_split을 바꿔가면서 최적의 값을 찾기때문에 이 두개의 매개변수를 동시에 바꿔가면서 최적의 값을 찾아야한다.
* for반복문으로 구현하여도 되지만 사이킷런에서 그리드 서치라는 도구를 사용하여 쉽게 할 수 있다.

###GridSearchCV클래스
* 해당 클래스는 하이퍼파라미터 탐색과 교차검증을 한번에 수행하기 때문에 따로 cross_validate()함수를 사용할 필요가 없다

In [9]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params,n_jobs=-1)


In [10]:
gs.fit(train_input,train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

* 사이킷런의 그리드 서치는 훈련이 끝나면 25개의 모델중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 모델을 다시 훈련하는데 이 모델은 gs객체의 best_estimator_ 속성에 저장되어있다.

In [11]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [12]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [13]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


* 넘파이 argmax()함수를 사용하면 가장 큰 값의 인덱스를 추출할 수 있다.

In [14]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


##정리
* 1.먼저 탐색할 매개변수를 지정한다.
* 2.그다음 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾는다(이 조합은 그리드 서치 객체에 저장됨).
* 3.그리드 서치는 최상의 매개변수에서(교차검증에 사용한 훈련세트가 아니라) 전체 훈련세트를 사용하여 최종모델을 훈련함.(이 모델도 그리드 서치 객체에 저장됨)

###min_impurity_decrease는 불순도 감소 최소량을 지정함
###min_samples_split으로 노드를 나누기 위한 최소 샘플수를 고름

In [15]:
params = {'min_impurity_decrease': np.arange(0.0001,0.001,0.0001), #첫번째 매개변수 값에서 시작하여 두번째 매개변수에 도달할 때까지 세번째 매개변수를 계속 더한 배열을 만듬
          'max_depth': range(5,20,1), #arange()함수와 비슷하지만 정수만 사용가능함
          'min_samples_split' : range(2,100,19)}

In [18]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)
print(gs.best_params_)#최상의 매개변수 조합
print(np.max(gs.cv_results_['mean_test_score']))#최상의 교차검증점수

{'max_depth': 11, 'min_impurity_decrease': 0.0004, 'min_samples_split': 2}
0.8681935292811135


###랜덤서치
* 매개변수 값이 수치일때 값의 범위나 간격을 미리 정하기 어려울때, 너무 많은 매개변수 조건이 있어서 그리드 서치 수행시간이 오래걸릴때 랜덤서치를 사용한다.

###싸이파이(scipy)라이브러리
* 파이썬의 핵심 과학 라이브러리(적분, 보간, 선형대수, 확률등을 포함한 수치계산 전용라이브러리)

In [19]:
from scipy.stats import uniform, randint

In [20]:
rgen = randint(0,10)
rgen.rvs(10)

array([7, 9, 2, 6, 3, 2, 2, 2, 7, 5])

In [21]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 90,  94, 105, 115, 103,  99,  92,  96, 100, 106]))

In [22]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.19037604, 0.32916909, 0.44768155, 0.40909573, 0.20051546,
       0.27198792, 0.09083604, 0.53090389, 0.7329349 , 0.8554272 ])

###min_samples_leaf매개변수
* 해당 매개변수는 리프 노드가 되기 위한 최소 샘플의 개수를 구해준다

In [24]:
params = {'min_impurity_decrease': uniform(0.0001,0.001),
          'max_depth':randint(20,50),
          'min_samples_split' : randint(2,25),
          'min_samples_leaf' : randint(1,25)}

In [25]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f57ea0b03a0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f57ea0b0220>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f57ea3f1cd0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f57ea0b0280>},
                   random_state=42)

In [28]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884


In [29]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
